# Hierarchical Clustering

**Hierarchical clustering** refers to a class of clustering methods that seek to build a **hierarchy** of clusters, in which some clusters contain others. In this assignment, we will explore a top-down approach, recursively bipartitioning the data using k-means.

## Import packages

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
import pandas as pd
import json

%matplotlib inline

## Load the Wikipedia dataset

In [9]:
wiki = pd.read_csv('people_wiki.csv')

As in the previous assignment, we extract the TF-IDF vector of each document.

For your convenience, we extracted the TF-IDF vectors from the dataset. The vectors are packaged in a sparse matrix, where the i-th row gives the TF-IDF vectors for the i-th document. Each column corresponds to a unique word appearing in the dataset.

To load in the TF-IDF vectors, run:

In [10]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')
map_index_to_word = pd.read_json('people_wiki_map_index_to_word.json', typ='series')

To be consistent with the k-means assignment, let's normalize all vectors to have unit norm.

In [17]:
tf_idf = normalize(tf_idf)
tf_idf.shape

(59071, 547979)

## Bipartition the Wikipedia dataset using k-means

Recall our workflow for clustering text data with k-means:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with some value of k.
4. Visualize the clustering results using the centroids, cluster assignments, and the original dataframe. We keep the original dataframe around because the data matrix does not keep auxiliary information (in the case of the text dataset, the title of each article).

Let us modify the workflow to perform bipartitioning:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with k=2.
4. Divide the data matrix into two parts using the cluster assignments.
5. Divide the dataframe into two parts, again using the cluster assignments. This step is necessary to allow for visualization.
6. Visualize the bipartition of data.

We'd like to be able to repeat Steps 3-6 multiple times to produce a **hierarchy** of clusters such as the following:
```
                      (root)
                         |
            +------------+-------------+
            |                          |
         Cluster                    Cluster
     +------+-----+             +------+-----+
     |            |             |            |
   Cluster     Cluster       Cluster      Cluster
```
Each **parent cluster** is bipartitioned to produce two **child clusters**. At the very top is the **root cluster**, which consists of the entire dataset.

Now we write a wrapper function to bipartition a given cluster using k-means. There are three variables that together comprise the cluster:

* `dataframe`: a subset of the original dataframe that correspond to member rows of the cluster
* `matrix`: same set of rows, stored in sparse matrix format
* `centroid`: the centroid of the cluster (not applicable for the root cluster)

Rather than passing around the three variables separately, we package them into a Python dictionary. The wrapper function takes a single dictionary (representing a parent cluster) and returns two dictionaries (representing the child clusters).

In [36]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    '''cluster: should be a dictionary containing the following keys
                * dataframe: original dataframe
                * matrix:    same data, in matrix format
                * centroid:  centroid for this particular cluster'''
    
    data_matrix = cluster['matrix']
    dataframe   = cluster['dataframe']
    
    # Run k-means on the data matrix with k=2. We use scikit-learn here to simplify workflow.
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs, random_state=seed, n_jobs=-1,verbose=2)
    kmeans_model.fit(data_matrix)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    # Divide the data matrix into two parts using the cluster assignments.
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], \
                                                      data_matrix[cluster_assignment==1]
    
    # Divide the dataframe into two parts, again using the cluster assignments.
    cluster_assignment_sa = np.array(cluster_assignment) # minor format conversion
    dataframe_left_child, dataframe_right_child     = dataframe[cluster_assignment_sa==0], \
                                                      dataframe[cluster_assignment_sa==1]
            
    # Package relevant variables for the child clusters
    cluster_left_child  = {'matrix': data_matrix_left_child,
                           'dataframe': dataframe_left_child,
                           'centroid': centroids[0]}
    cluster_right_child = {'matrix': data_matrix_right_child,
                           'dataframe': dataframe_right_child,
                           'centroid': centroids[1]}
    
    return cluster_left_child, cluster_right_child

The following cell performs bipartitioning of the Wikipedia dataset. Allow 5 minutes to finish.

Note. For the purpose of the assignment, we set an explicit seed (`seed=1`) to produce identical outputs for every run. In pratical applications, you might want to use different random seeds for all runs.

In [44]:
%%time
wiki_data = {'matrix': tf_idf, 'dataframe': wiki} # no 'centroid' for the root cluster
# reduced the number of iterations from 100 to 5 as 5 already takes 5 minutes
left_child, right_child = bipartition(wiki_data, maxiter=5, num_runs=6, seed=1)

CPU times: user 290 ms, sys: 122 ms, total: 412 ms
Wall time: 4min 43s


Let's examine the contents of one of the two clusters, which we call the `left_child`, referring to the tree visualization above.

In [45]:
left_child

{'matrix': <25533x547979 sparse matrix of type '<class 'numpy.float64'>'
 	with 4280739 stored elements in Compressed Sparse Row format>,
 'dataframe':                                                      URI  \
 0            <http://dbpedia.org/resource/Digby_Morrell>   
 13     <http://dbpedia.org/resource/Anthony_Gueterboc...   
 14      <http://dbpedia.org/resource/David_Chernushenko>   
 17     <http://dbpedia.org/resource/Paddy_Dunne_(Gael...   
 19     <http://dbpedia.org/resource/John_Angus_Campbell>   
 ...                                                  ...   
 59064            <http://dbpedia.org/resource/Oz_Bengur>   
 59065  <http://dbpedia.org/resource/Dee_Brown_(basket...   
 59068  <http://dbpedia.org/resource/David_Cass_(footb...   
 59069          <http://dbpedia.org/resource/Keith_Elias>   
 59070         <http://dbpedia.org/resource/Fawaz_Damrah>   
 
                                           name  \
 0                                Digby Morrell   
 13     Antho

And here is the content of the other cluster we named `right_child`.

In [46]:
right_child

{'matrix': <33538x547979 sparse matrix of type '<class 'numpy.float64'>'
 	with 6098544 stored elements in Compressed Sparse Row format>,
 'dataframe':                                                      URI                 name  \
 1           <http://dbpedia.org/resource/Alfred_J._Lewy>       Alfred J. Lewy   
 2            <http://dbpedia.org/resource/Harpdog_Brown>        Harpdog Brown   
 3      <http://dbpedia.org/resource/Franz_Rottensteiner>  Franz Rottensteiner   
 4                   <http://dbpedia.org/resource/G-Enka>               G-Enka   
 5            <http://dbpedia.org/resource/Sam_Henderson>        Sam Henderson   
 ...                                                  ...                  ...   
 59058        <http://dbpedia.org/resource/George_Krause>        George Krause   
 59059           <http://dbpedia.org/resource/Sean_Slade>           Sean Slade   
 59060         <http://dbpedia.org/resource/Lynne_Lipton>         Lynne Lipton   
 59066           <http://dbpe

## Visualize the bipartition

We provide you with a modified version of the visualization function from the k-means assignment. For each cluster, we print the top 5 words with highest TF-IDF weights in the centroid and display excerpts for the 8 nearest neighbors of the centroid.

In [47]:
def display_single_tf_idf_cluster(cluster, map_index_to_word):
    '''map_index_to_word: dataframe specifying the mapping betweeen words and column indices'''
    
    wiki_subset   = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid      = cluster['centroid']
    
    # Print top 5 words with largest TF-IDF weights in the cluster
    idx = centroid.argsort()[::-1]
    for i in range(5):
        print('{0:s}:{1:.3f}'.format(map_index_to_word.index[idx[i]], centroid[idx[i]])),
    print('')
    
    # Compute distances from the centroid to all data points in the cluster.
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    # compute nearest neighbors of the centroid within the cluster.
    nearest_neighbors = distances.argsort()
    # For 8 nearest neighbors, print the title as well as first 180 characters of text.
    # Wrap the text at 80-character mark.
    for i in range(8):
        text = ' '.join(wiki_subset.iloc[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print('* {0:50s} {1:.5f}\n  {2:s}\n  {3:s}'.format(wiki_subset.iloc[nearest_neighbors[i]]['name'],
              distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
    print('')

Let's visualize the two child clusters:

In [48]:
display_single_tf_idf_cluster(left_child, map_index_to_word)

19771992according:0.018
gan:0.016
sibinki:0.016
gonino:0.014
anchoragearea:0.013

* Todd Williams                                      0.97309
  todd michael williams born february 13 1971 in syracuse new york is a former major league 
  baseball relief pitcher he attended east syracuseminoa high school
* Micky Adams                                        0.97447
  michael richard micky adams born 8 november 1961 is an english former professional footbal
  ler turned football manager who is in charge of league two side
* Justin Knoedler                                    0.97453
  justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
  baseball catcherknoedler was originally drafted by the st louis cardinals
* Phil King (footballer)                             0.97473
  philip geoffrey king born 28 december 1967 is an english former professional footballer he
   represented england at under21 level and in a b international he
* Ashley Prescott      

In [49]:
display_single_tf_idf_cluster(right_child, map_index_to_word)

serieslong:0.037
bostonas:0.025
33story:0.017
allmvfc:0.015
conder:0.011

* Madonna (entertainer)                              0.96656
  madonna louise ciccone tkoni born august 16 1958 is an american singer songwriter actress 
  and businesswoman she achieved popularity by pushing the boundaries of lyrical
* Janet Jackson                                      0.96672
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Anita Kunz                                         0.96782
  anita e kunz oc born 1956 is a canadianborn artist and illustratorkunz has lived in london
   new york and toronto contributing to magazines and working
* Cher                                               0.96997
  cher r born cherilyn sarkisian may 20 1946 is an american singer actress and television ho
  st described as embodying female autonomy in a maledominated industry
* Alexandra Potter             

The right cluster consists of artists (singers and actors/actresses), whereas the left cluster consists of football/baseball players. So far, we have a single-level hierarchy consisting of two clusters, as follows:

```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
          Football/Baseball players                              Artists
```

Is this hierarchy good enough? **When building a hierarchy of clusters, we must keep our particular application in mind.** For instance, we might want to build a **directory** for Wikipedia articles. A good directory would let you quickly narrow down your search to a small set of related articles. The categories of football/baseball players and artists are too general to facilitate efficient search. For this reason, we decide to build another level into our hierarchy of clusters with the goal of getting more specific cluster structure at the lower level. To that end, we subdivide both clusters.

## Perform recursive bipartitioning

### Cluster of football/baseball players and artists

To help identify the clusters we've built so far, let's give them easy-to-read aliases:

In [59]:
football_baseball_players = left_child
artists                   = right_child

Using the bipartition function, we produce two child clusters of the athlete cluster:

In [61]:
%%time
# Bipartition the cluster of football_baseball_players
left_child_football_baseball_players, right_child_football_baseball_players = bipartition(
    athletes, maxiter=5, num_runs=6, seed=1)

The left child cluster mainly consists of athletes:

In [62]:
display_single_tf_idf_cluster(left_child_football_baseball_players, map_index_to_word)

anchoragearea:0.034
gonino:0.029
sibinki:0.029
19771992according:0.025
ssls:0.024

* Jason Roberts (footballer)                         0.95786
  jason andre davis roberts mbe born 25 january 1978 is a former professional footballer and
   now a football punditborn in park royal london roberts was
* Ashley Prescott                                    0.95797
  ashley prescott born 11 september 1972 is a former australian rules footballer he played w
  ith the richmond and fremantle football clubs in the afl between
* Chris Day                                          0.95841
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Sol Campbell                                       0.95889
  sulzeer jeremiah sol campbell born 18 september 1974 is a former england international foo
  tballer a central defender he had a 19year career playing in the
* Todd Curley                  

On the other hand, the right child cluster consists mainly of politicians:

In [63]:
display_single_tf_idf_cluster(right_child_football_baseball_players, map_index_to_word)

19771992according:0.086
qc:0.075
sibinki:0.050
guitarscordray:0.045
ibnez:0.042

* Todd Williams                                      0.91292
  todd michael williams born february 13 1971 in syracuse new york is a former major league 
  baseball relief pitcher he attended east syracuseminoa high school
* Justin Knoedler                                    0.91354
  justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
  baseball catcherknoedler was originally drafted by the st louis cardinals
* Steve Springer                                     0.91414
  steven michael springer born february 11 1961 is an american former professional baseball 
  player who appeared in major league baseball as a third baseman and
* Dave Ford                                          0.91434
  david alan ford born december 29 1956 is a former major league baseball pitcher for the ba
  ltimore orioles born in cleveland ohio ford attended lincolnwest
* Kevin Nicholson (ba

Our hierarchy of clusters now looks like this:
```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
        Football/Baseball players                                Artists
                    +
                    |
         +----------+----------+
         |                     |
         |                     |
         +                     |
    Footballers        Baseball players
```

Should we keep subdividing the clusters? If so, which cluster should we subdivide? To answer this question, we again think about our application. Since we organize our directory by topics, it would be nice to have topics that are about as coarse as each other. For instance, if one cluster is about baseball, we expect some other clusters about football, basketball, volleyball, and so forth. That is, **we would like to achieve similar level of granularity for all clusters.**

Both the athletes and artists node can be subdivided more, as each one can be divided into more descriptive professions (singer/actress/painter/director, or baseball/football/basketball, etc.). Let's explore subdividing the athletes cluster further to produce finer child clusters.

Let's give the clusters aliases as well:

In [64]:
footballers       = left_child_football_baseball_players
baseball_players  = right_child_football_baseball_players

### Cluster of footballers

In answering the following quiz question, take a look at the topics represented in the top documents (those closest to the centroid), as well as the list of words with highest TF-IDF weights.

Let us bipartition the cluster of footballers.

In [65]:
%%time
left_child_footballers, right_child_footballers = bipartition(footballers, maxiter=10, num_runs=6, seed=1)

CPU times: user 91.1 ms, sys: 153 ms, total: 244 ms
Wall time: 1min 26s


In [66]:
display_single_tf_idf_cluster(left_child_footballers, map_index_to_word)
display_single_tf_idf_cluster(right_child_footballers, map_index_to_word)

ebe:0.037
qafl:0.033
agency:0.032
spanky:0.027
addaction:0.027

* Brian Davis (golfer)                               0.94927
  brian lester davis born 2 august 1974 is an english professional golferdavis was born in l
  ondon he turned professional in 1994 and became a member
* Tim Clark (golfer)                                 0.94939
  timothy henry clark born 17 december 1975 is a south african professional golfer who curre
  ntly plays on the pga tour his biggest win to date
* Philip Parkin                                      0.95140
  andrew philip parkin born 12 december 1961 is a welsh professional golfer who has also wor
  ked as a golf commentator and analystparkin was born in
* Marc Leishman                                      0.95204
  marc leishman born 24 october 1983 is an australian professional golfer who currently play
  s on the pga tour in 2009 he won the rookie of
* Craig Parry                                        0.95216
  craig david parry born 12 january 1966

**Quiz Question**. Which diagram best describes the hierarchy right after splitting the `footballers` cluster? Refer to the quiz form for the diagrams.

**Caution**. The granularity criteria is an imperfect heuristic and must be taken with a grain of salt. It takes a lot of manual intervention to obtain a good hierarchy of clusters.

* **If a cluster is highly mixed, the top articles and words may not convey the full picture of the cluster.** Thus, we may be misled if we judge the purity of clusters solely by their top documents and words. 
* **Many interesting topics are hidden somewhere inside the clusters but do not appear in the visualization.** We may need to subdivide further to discover new topics. For instance, subdividing the `football` cluster led to the appearance of golfers.

### Cluster of non-athletes

Now let us subdivide the cluster of artists.

In [67]:
%%time 
# Bipartition the cluster of non-athletes
left_child_artists, right_child_artists = bipartition(artists, maxiter=5, num_runs=6, seed=1)

CPU times: user 215 ms, sys: 103 ms, total: 318 ms
Wall time: 2min 44s


In [68]:
display_single_tf_idf_cluster(left_child_artists, map_index_to_word)

serieslong:0.131
bostonas:0.085
allmvfc:0.012
interlingual:0.012
33story:0.011

* Janet Jackson                                      0.93698
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Lauren Royal                                       0.93725
  lauren royal born march 3 circa 1965 is a book writer from california royal has written bo
  th historic and novelistic booksa selfproclaimed angels baseball fan
* Barbara Hershey                                    0.93795
  barbara hershey born barbara lynn herzstein february 5 1948 once known as barbara seagull 
  is an american actress in a career spanning nearly 50 years
* Jane Fonda                                         0.93999
  jane fonda born lady jayne seymour fonda december 21 1937 is an american actress writer po
  litical activist former fashion model and fitness guru she is
* Janine Shepherd                          

In [69]:
display_single_tf_idf_cluster(right_child_artists, map_index_to_word)

33story:0.018
allmvfc:0.016
gan:0.012
efovi:0.011
scientistagreed:0.011

* Julian Knowles                                     0.97393
  julian knowles is an australian composer and performer specialising in new and emerging te
  chnologies his creative work spans the fields of composition for theatre dance
* Wilson McLean                                      0.97596
  wilson mclean born 1937 is a scottish illustrator and artist he has illustrated primarily 
  in the field of advertising but has also provided cover art
* Craig Pruess                                       0.97624
  craig pruess born 1950 is an american composer musician arranger and gold platinum record 
  producer who has been living in britain since 1973 his career
* Peter Combe                                        0.97640
  peter combe born 20 october 1948 is an australian childrens entertainer and musicianmusica
  l genre childrens musiche has had 22 releases including seven gold albums two
* Brenton Broadstock    

The clusters are not as clear, but the left cluster has a tendency to show important female figures, and the right one to show composers.

Let's divide them further.

In [70]:
female_figures = left_child_artists
composers = right_child_artists

**Quiz Question**. Let us bipartition the clusters `female_figures` and `composers`. Which diagram best describes the resulting hierarchy of clusters for artists? Refer to the quiz for the diagrams.

**Note**. Use `maxiter=100, num_runs=6, seed=1` for consistency of output.

In [72]:
left_child_female_figures, right_child_female_figures = bipartition(female_figures, maxiter=5, num_runs=6, seed=1)
display_single_tf_idf_cluster(left_child_female_figures, map_index_to_word)
display_single_tf_idf_cluster(right_child_female_figures, map_index_to_word)

serieslong:0.131
bostonas:0.085
allmvfc:0.012
interlingual:0.012
33story:0.011

* Janet Jackson                                      0.93650
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Lauren Royal                                       0.93689
  lauren royal born march 3 circa 1965 is a book writer from california royal has written bo
  th historic and novelistic booksa selfproclaimed angels baseball fan
* Barbara Hershey                                    0.93748
  barbara hershey born barbara lynn herzstein february 5 1948 once known as barbara seagull 
  is an american actress in a career spanning nearly 50 years
* Jane Fonda                                         0.93960
  jane fonda born lady jayne seymour fonda december 21 1937 is an american actress writer po
  litical activist former fashion model and fitness guru she is
* Janine Shepherd                          

In [73]:
left_child_composers, right_child_composers = bipartition(composers, maxiter=5, num_runs=6, seed=1)
display_single_tf_idf_cluster(left_child_composers, map_index_to_word)
display_single_tf_idf_cluster(right_child_composers, map_index_to_word)

33story:0.055
conder:0.036
nanri:0.034
burkewhite:0.021
1975one:0.020

* Brenton Broadstock                                 0.95824
  brenton broadstock ao born 1952 is an australian composerbroadstock was born in melbourne 
  he studied history politics and music at monash university and later composition
* Julian Knowles                                     0.96174
  julian knowles is an australian composer and performer specialising in new and emerging te
  chnologies his creative work spans the fields of composition for theatre dance
* Prince (musician)                                  0.96176
  prince rogers nelson born june 7 1958 known by his mononym prince is an american singerson
  gwriter multiinstrumentalist and actor he has produced ten platinum albums
* Tom Bancroft                                       0.96203
  tom bancroft born 1967 london is a british jazz drummer and composer he began drumming age
  d seven and started off playing jazz with his father
* Will.i.am      